<span style="font-size:1.1em;">Colab'a Google drive'ı entegre ediyoruz. Kullanılacak olan veriseti Google Drive'da bulunmaktadır</span>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

<span style="font-size:1.1em;">Google Drive'ımızın root pathi</span> ```gdrive/My Drive``` <span style="font-size:1.1em;">oluyor. Proje için gerekli verisetini **mbti** adında bir klasör oluşturup içerisine yüklüyoruz. İlgili verisetinin pathi</span> ```gdrive/My Drive/mbti/all_users.csv``` <span style="font-size:1.1em;">oluyor.</span>

In [0]:
PREPROCESSED_DATASET_WITH_STEMMER = "gdrive/My Drive/mbti/preprocessed_dataset_with_stemming.csv"
PREPROCESSED_DATASET_WITHOUT_STEMMER = "gdrive/My Drive/mbti/preprocessed_dataset_no_stemming.csv"
PREPROCESSED_DATASET_ZEMBEREK = "gdrive/My Drive/mbti/preprocessed_dataset_zemberek"
TRIMMED_DATASET = "gdrive/My Drive/mbti/trimmed_dataset.csv"
RAW_DATASET = "gdrive/My Drive/mbti/all_users_v2.csv"

<span style="font-size:1.1em;">Hangi veriseti kullanılarak işlem yapılacaksa yukardaki pathlerden biri seçilir ve parametre olarak verilir.</span>

In [0]:
import pandas as pd 
df = pd.read_csv(PREPROCESSED_DATASET_WITHOUT_STEMMER, sep = ';', header = 0)

In [0]:
df

Her tipten eşit sayıda entry alınabilmesi adına öncelikle bir sınıftaki en az olan entry sayısı bulunur. Her boyut için geçerli olacak minimum entry sayısı bulunur. Buradaki amacımız subsampling yaparak SVM için harcanan süreyi azaltabilmektir.

In [0]:
entry_counts = []
entry_counts.append(df.groupby('typeClass', as_index = False).count().min().entry)
entry_counts.append(df.groupby('I/E', as_index = False).count().min().entry)
entry_counts.append(df.groupby('S/N', as_index = False).count().min().entry)
entry_counts.append(df.groupby('T/F', as_index = False).count().min().entry)
entry_counts.append(df.groupby('J/P', as_index = False).count().min().entry)

min_entry = min(entry_counts)


Eşit sayıda entry olacak şekilde yeni bir dataframe olusturulur

In [0]:
analysts_df = df[df['typeClass'] == 'analysts']
analysts_df = analysts_df.iloc[0 : min_entry]

analysts_df.shape[0]


In [0]:
explorers_df = df[df['typeClass'] == 'explorers']
explorers_df = explorers_df.iloc[0 : min_entry]

explorers_df.shape[0]

In [0]:
sentinels_df = df[df['typeClass'] == 'sentinels']
sentinels_df = sentinels_df.iloc[0 : min_entry]

sentinels_df.shape[0]

In [0]:
diplomats_df = df[df['typeClass'] == 'diplomats']

diplomats_df = diplomats_df.iloc[0 : min_entry]

diplomats_df.shape[0]

Yeni bir dataframe oluşturulur.

In [0]:
equal_entries_df = pd.concat([diplomats_df, sentinels_df, explorers_df, analysts_df]).reset_index(drop=True)
equal_entries_df

Oluşturulan dataframe shuffle edilir.

In [0]:
equal_entries_df = equal_entries_df.sample(frac=1).reset_index(drop=True)

In [0]:
equal_entries_df

<span style="font-size:1.1em;">TF-IDF özellik vektörünün çıkartılmasında kullanılacak değişken aşağıda belirlenmiş olan parametrelerle oluşturulur.</span>

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True, norm='l2', encoding='utf-8', ngram_range=(1, 2), max_features=100, analyzer = 'word', token_pattern=r'\w{1,}')

**Feature Extraction** 

Burada tf-idf vektörlerinin çıkartılmasında kullanılacak olan vocabulary oluşturulur.

In [0]:
import numpy as np

equal_entries_df['entry'] = equal_entries_df['entry'].apply(lambda x: np.str_(x)) # ValueError: np.nan is an invalid document seklinde bir hata verdigi icin bunu asmak adina yapildi.

In [0]:
tfidf_vectorizer.fit(equal_entries_df['entry'])

Veriseti train ve test olmak üzere ikiye ayrılır. Test %20 ve train %80'ini oluşturacak şekilde tüm veriseti bölünür. random_state parametresi ile tekrardan bölündüğünde bir öncekiyle aynı train ve test veri setlerinin oluşturulması sağlanır.

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['typeClass'], random_state = 42, test_size = 0.20, shuffle = True)

y_actual = y_test.values

In [0]:
X_train.shape[0]

In [0]:
X_test.shape[0]

Train ve test datasetlerinden tf-idf vektörleri çıkartılır


In [0]:
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [0]:
X_train_tfidf

In [0]:
X_test_tfidf

SVM kullanılarak tahmin edilir.

In [0]:
from sklearn import svm
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf, y_train)

In [0]:
y_predicted = SVM.predict(X_test_tfidf)

In [0]:
y_predicted

<span style="font-size:1.1em">Yapılacak tahminlerle ilgili istatistiksel verileri tutmak için</span> ```predictions_result```<span style="font-size:1.1em"> adında bir değişken oluşturulur.</span>

<span style="font-size:1.1em">Bu değişkenin yapısı aşağıdaki gibidir.</span>

```json
{
    "predicted": {
        "analysts":  { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "diplomats": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "explorers": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "sentinels": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
    }
}
```

* <span style="font-size:1.1em;">Yapılan tahminlerle ilgili verilere ulaşabilmek için</span>

    ```predictions_results['predicted']```


* <span style="font-size:1.1em;">Yapılan tahminin analyst ise:</span>

    ```predictions_results['predicted']['analysts']``` 


* <span style="font-size:1.1em;">Yapılan analyst tahmininin gerçek değerlerine erişmek için:</span>     

    ```predictions_results['predicted']['analysts']['actual']```  


* <span style="font-size:1.1em;">Test verisi, model tarafından analysts olarak tahmin edilmiştir ve bu verinin gerçek değeri de analysts'tir.</span>

    ```predictions_results['predicted']['analysts']['actual']['analysts']``` 

In [0]:
prediction_results = {'predicted': {}}  ## prediction_result['analysts'] means prediction is 'analysts'

prediction_results['predicted']['analysts']  = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['diplomats'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['explorers'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['sentinels'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}

## prediction_result['analysts']['diplomats'] means prediction is analysts but actual value is diplomats

```prediction_results```<span style="font-size:1.1em"> içerisinde tutulan sayaçların değerleri arttırılır.</span>

In [0]:
for i in range(len(y_predicted)):
  predicted_value = y_predicted[i]
  actual_value = y_actual[i]
  prediction_results['predicted'][predicted_value]['actual'][actual_value] += 1

<span style="font-size:1.1em">JSON formatına çevrilir </span>```dict``` <span style="font-size:1.1em">tipi. Bu sayede daha okunaklı bir şekilde print edilmiş olur. </span>

In [0]:
import json

print(json.dumps(prediction_results, indent = 2))

<span style="font-size:1.1em;">İlgili field extract edilir</span> ```dict``` <span style="font-size:1.1em;">yapısından.</span>

In [0]:
results = prediction_results['predicted']

<span style="font-size:1.1em;">Başarı oranı hesaplanır</span>

In [0]:
accuracy = (results['analysts']['actual']['analysts'] + results['diplomats']['actual']['diplomats'] + results['explorers']['actual']['explorers'] + results['sentinels']['actual']['sentinels']) / len(y_predicted)
accuracy

<span style="font-size:1.1em;">**E/I** boyutu tahmin edilir</span>

Eşit sayıda entry olacak şekilde yeni bir dataframe olusturulur

In [0]:
I_df = df[df['I/E'] == 'I']
I_df = I_df.iloc[0 : min_entry]

I_df.shape[0]


In [0]:
E_df = df[df['I/E'] == 'E']

E_df = E_df.iloc[0 : min_entry]

E_df.shape[0]

Yeni bir dataframe oluşturulur.

In [0]:
equal_entries_df = pd.concat([I_df, E_df]).reset_index(drop=True)
equal_entries_df

In [0]:
equal_entries_df['entry'] = equal_entries_df['entry'].apply(lambda x: np.str_(x)) # ValueError: np.nan is an invalid document seklinde bir hata verdigi icin bunu asmak adina yapildi.

X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['I/E'], random_state = 42, shuffle = True)  ## Geri kalanlar S, T, J

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [0]:
y_predicted

In [0]:
predicted = {}
predicted['I'] = {'actual': {'I': 0, 'E': 0}}
predicted['E'] = {'actual': {'I': 0, 'E': 0}}
predicted

In [0]:
for i in range(len(y_predicted)):
  predicted[y_predicted[i]]['actual'][y_actual[i]] += 1


In [0]:
predicted

In [0]:
accuracy = (predicted['E']['actual']['E'] + predicted['I']['actual']['I']) / len(y_predicted)

In [0]:
accuracy

<span style="font-size:1.1em">**S/N** boyutu tahmin edilir.</span>

Her tipten eşit sayıda entry alınabilmesi adına öncelikle bir sınıftaki en az olan entry sayısı bulunur

In [0]:

min_entry


Eşit sayıda entry olacak şekilde yeni bir dataframe olusturulur

In [0]:
S_df = df[df['S/N'] == 'S']
S_df = S_df.iloc[0 : min_entry]

S_df.shape[0]


In [0]:
N_df = df[df['S/N'] == 'N']

N_df = N_df.iloc[0 : min_entry]

N_df.shape[0]

Yeni bir dataframe oluşturulur.

In [0]:
equal_entries_df = pd.concat([S_df, N_df]).reset_index(drop=True)
equal_entries_df

In [0]:
equal_entries_df['entry'] = equal_entries_df['entry'].apply(lambda x: np.str_(x)) # ValueError: np.nan is an invalid document seklinde bir hata verdigi icin bunu asmak adina yapildi.

X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['S/N'], random_state = 42, shuffle = True)  ## Geri kalan boyutlar: T, J

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [0]:
predicted['N'] = {'actual': {'N': 0, 'S': 0}}
predicted['S'] = {'actual': {'N': 0, 'S': 0}}

predicted

In [0]:
for i in range(len(y_predicted)):
  predicted[y_predicted[i]]['actual'][y_actual[i]] += 1

In [0]:
predicted

In [0]:
accuracy = (predicted['N']['actual']['N'] + predicted['S']['actual']['S']) / len(y_predicted)

In [0]:
accuracy

<span style="font-size:1.1em">**T/F** boyutu tahmin edilir.</span>

Eşit sayıda entry olacak şekilde yeni bir dataframe olusturulur

In [0]:
T_df = df[df['T/F'] == 'T']
T_df = T_df.iloc[0 : min_entry]

T_df.shape[0]


In [0]:
F_df = df[df['T/F'] == 'F']

F_df = F_df.iloc[0 : min_entry]

F_df.shape[0]

Yeni bir dataframe oluşturulur.

In [0]:
equal_entries_df = pd.concat([T_df, F_df]).reset_index(drop=True)
equal_entries_df

In [0]:
equal_entries_df['entry'] = equal_entries_df['entry'].apply(lambda x: np.str_(x)) # ValueError: np.nan is an invalid document seklinde bir hata verdigi icin bunu asmak adina yapildi.

X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['T/F'], random_state = 42, shuffle = True)  ##  J

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [0]:
predicted['T'] = {'actual': {'T': 0, 'F': 0}}
predicted['F'] = {'actual': {'T': 0, 'F': 0}}

predicted

In [0]:
for i in range(len(y_predicted)):
  predicted[y_predicted[i]]['actual'][y_actual[i]] += 1

In [0]:
predicted

In [0]:
accuracy = (predicted['F']['actual']['F'] + predicted['T']['actual']['T']) / len(y_predicted)

In [0]:
accuracy

<span style="font-size:1.1em">**J/P** boyutu tahmin edilir.</span>

Eşit sayıda entry olacak şekilde yeni bir dataframe olusturulur

In [0]:
J_df = df[df['J/P'] == 'J']
J_df = J_df.iloc[0 : min_entry]

J_df.shape[0]


In [0]:
P_df = df[df['J/P'] == 'P']

P_df = P_df.iloc[0 : min_entry]

P_df.shape[0]

Yeni bir dataframe oluşturulur.

In [0]:
equal_entries_df = pd.concat([I_df, E_df]).reset_index(drop=True)
equal_entries_df

In [0]:
equal_entries_df['entry'] = equal_entries_df['entry'].apply(lambda x: np.str_(x)) # ValueError: np.nan is an invalid document seklinde bir hata verdigi icin bunu asmak adina yapildi.

X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['J/P'], random_state = 42, shuffle = True) 

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [0]:
predicted['J'] = {'actual': {'J': 0, 'P': 0}}
predicted['P'] = {'actual': {'J': 0, 'P': 0}}

predicted

In [0]:
for i in range(len(y_predicted)):
  predicted[y_predicted[i]]['actual'][y_actual[i]] += 1

In [0]:
predicted

In [0]:
accuracy = (predicted['P']['actual']['P'] + predicted['J']['actual']['J']) / len(y_predicted)

In [0]:
accuracy

In [0]:
prediction_results['predicted'].update(predicted) 

prediction_results